## Taller Unidad 8
### OpenMp - Multiprocessing

#### Solución:

#### 1. ¿Qué es la programación paralela y cómo se relaciona con: la biblioteca multiprocessing, y con OpenMP?
 - La programación paralela es un paradigma de la programación que permite la ejecución simultánea de varías tareas en múltiples unidades de procesamiento. 
 - Relación con la biblioteca multiprocessing: Esta biblioteca permite aprovechar la capacidad de multiprocesamiento en sistemas con múltiples núcleos de CPU o procesadores, brindadndo la capacidad para crear y gestionar procesos paralelos, permitiendo distribuir el trabajo entre ellos y combinar los resultados al final.
 - Relación con OpenMp: OpenMP es una interfaz de programación de aplicaciones para programación paralela basada en memoria compartida, esta permite agregar pragmas al código fuente para indicar qué regiones del código se deben paralelizar, donde ya el compilador se encarga de generar el código paralelo correspondiente facilitando el proceso.

 - Basicamente tanto la biblioteca multiprocessing como OpenMP permiten aprovechar la capacidad de multiprocesamiento en sistemas modernos, pero difieren en su enfoque y nivel de abstracción. Multiprocessing es una biblioteca de Python que facilita la creación de procesos paralelos, mientras que OpenMP es una API de más bajo nivel que permite paralelizar regiones de código utilizando directivas en lenguajes como C.

#### 2. Diferencias entre procesos y subprocesos.
##### Procesos
- Un proceso es una instancia de un programa en ejecución, con su propio espacio de direcciones de memoria.
- Los procesos son independientes entre sí y no comparten memoria directamente.
- La creación de un nuevo proceso requiere más recursos del sistema (memoria, archivos, etc.) que la creación de un subproceso.
- La comunicación entre procesos se realiza mediante mecanismos como tuberías, sockets, señales o memoria compartida.
- Si un proceso falla, no afecta directamente a otros procesos.
- Cada proceso tiene su propio ciclo de planificación independiente.

##### Subprocesos o threads
- Un subproceso es una ruta de ejecución dentro de un mismo proceso.
- Los subprocesos comparten el mismo espacio de direcciones de memoria del proceso al que pertenecen.
- La creación de un nuevo subproceso es más ligera y requiere menos recursos que la creación de un proceso.
- La comunicación y sincronización entre subprocesos se realiza a través de estructuras de datos compartidas y mecanismos de sincronización como semáforos, bloqueos, etc.
- Si un subproceso falla, puede afectar a otros subprocesos dentro del mismo proceso.
- Los subprocesos son planificados por el mismo planificador de procesos del sistema operativo.

En pocas palabras los procesos son instancias independientes de un programa en ejecución, mientras que los subprocesos son rutas de ejecución dentro de un mismo proceso, compartiendo recursos como memoria, donde los subprocesos son más livianos y permiten un mayor grado de paralelismo dentro de un mismo proceso, pero también son más propensos a interferencias y errores

#### 3. ¿Cómo utilizar multiprocessing para dividir una tarea en múltiples procesos?
1. Identificar la tarea a paralelizar.
2. Importar la biblioteca multiprocessing. 
3. Definir la función a ejecutar en paralelo.
4. Dividir los datos de entrada: Antes de comenzar el procesamiento paralelo se necesita dividir los datos de entrada en partes más pequeña, cada parte será procesada por un proceso diferente.
5. Crear un pool: La clase Pool de multiprocessing es la encargada de gestionar un grupo de procesos de trabajo, se crea un objeto Pool especificando el número de procesos que se desea utilizar, en caso de no tener seguridad en esto se puede usar None para utilizar tantos procesos como núcleos de CPU estén disponibles.
6. Aplicar la función paralela: Utilizar el método map o imap del objeto Pool para aplicar tu función a cada parte de los datos de entrada de forma paralela. Cada proceso ejecutará tu función con una parte diferente de los datos.
7. Esperar a que los procesos terminen. Se pueden utilizar los métodos close() y join() del objeto Pool para hacer esto.
8. Coombinar los resultados.
9. Cerrar el objeto Pool.

Ejemplo en el siguiente campo 

In [4]:
# 2. Importa la biblioteca multiprocessing (ya hecho al inicio)
import multiprocessing

# 3. Definir la función a ejecutar en paralelo
def suma_cuadrados(numeros):
    resultado = sum(x**2 for x in numeros)
    return resultado

# 4. Dividir los datos de entrada
def dividir_datos(data, num_procesos):
    chunk_size = len(data) // num_procesos
    return [data[i*chunk_size:(i+1)*chunk_size] for i in range(num_procesos)]

if __name__ == '__main__':
     # 1. Identifica la tarea a paralelizar (calcular la suma de los cuadrados de una lista de números)

    # Datos de entrada
    numeros = list(range(1, 101)) 

    # 5. Crear un objeto Pool
    num_procesos = multiprocessing.cpu_count()  
    pool = multiprocessing.Pool(processes=num_procesos)

     # 4. Dividir los datos de entrada (usando la función dividir_datos)
    datos_divididos = dividir_datos(numeros, num_procesos)

    # 6. Aplicar la función paralela
    resultados = pool.map(suma_cuadrados, datos_divididos)

    # 7. Esperar a que los procesos terminen
    pool.close()
    pool.join()

    # 8. Combinar los resultados
    resultado_final = sum(resultados)

    print(f"La suma de los cuadrados de los números del 1 al 100 es: {resultado_final}")

     # 9. Cerrar el objeto Pool (ya se hizo en el paso 7)

La suma de los cuadrados de los números del 1 al 100 es: 299536


#### 4. Actividad práctica: suma de números enteros.
1.  Utilizar multiprocessing para acelerar el cálculo de la suma de los primeros N números enteros.
2.  Comparar el tiempo de ejecución de la versión secuencial con la versión paralela.
3.  Experimentar con diferentes valores para el número de procesos para ver cómo afecta el rendimiento.
4.  Crear una tabla de comparación.


In [13]:
import multiprocessing  # Importa el módulo multiprocessing para trabajar con procesos en paralelo
import time  # Importa el módulo time para medir el tiempo de ejecución
import pandas as pd  # Importa el módulo pandas para trabajar con datos tabulares

def suma_rango(rango):
    return sum(range(rango[0], rango[1]))  # Función que suma los números dentro de un rango

def suma_paralela(n, num_procesos):
    pool = multiprocessing.Pool(processes=num_procesos)  # Crea un grupo de procesos paralelos
    rango_por_proceso = n // num_procesos  # Calcula el rango de enteros a procesar por cada proceso
    rangos = [(i*rango_por_proceso, (i+1)*rango_por_proceso) for i in range(num_procesos)]  # Divide los enteros en rangos
    resultados = pool.map(suma_rango, rangos)  # Aplica la función suma_rango a cada rango utilizando el grupo de procesos
    pool.close()  # Cierra el grupo de procesos
    pool.join()  # Espera a que todos los procesos en el grupo finalicen
    return sum(resultados)  # Devuelve la suma total de los resultados parciales obtenidos por cada proceso

def suma_secuencial(n):
    return sum(range(n))  # Función que calcula la suma de los primeros n enteros de manera secuencial

if __name__ == '__main__':
    n = int(input("Ingrese el número de enteros a sumar: "))  # Solicita al usuario ingresar el número de enteros a sumar
    num_procesos = int(input("Ingrese el número de procesos: "))  # Solicita al usuario ingresar el número de procesos

    datos = []  # Lista para almacenar los datos de tiempo de ejecución

    for i in range(1, num_procesos + 1):  # Itera sobre diferentes cantidades de procesos
        # Versión secuencial
        inicio = time.time()  # Marca el tiempo de inicio
        resultado_secuencial = suma_secuencial(n)  # Calcula la suma secuencial de los primeros n enteros
        tiempo_secuencial = time.time() - inicio  # Calcula el tiempo transcurrido

        # Versión paralela
        inicio = time.time()  # Marca el tiempo de inicio
        resultado_paralelo = suma_paralela(n, i)  # Calcula la suma paralela de los primeros n enteros con i procesos
        tiempo_paralelo = time.time() - inicio  # Calcula el tiempo transcurrido

        # Agregar fila a la lista de datos
        datos.append({"Número de procesos": i,
                      "Tiempo secuencial (s)": tiempo_secuencial,
                      "Tiempo paralelo (s)": tiempo_paralelo})  # Agrega los tiempos a la lista de datos

    # Crear DataFrame de Pandas
    tabla = pd.DataFrame(datos)  # Crea un DataFrame a partir de la lista de datos

    # Imprimir tabla con título que indica el valor de n
    print(f"Resultados para sumar los primeros {n} enteros con diferentes números de procesos:\n")  # Imprime un mensaje indicando el valor de n
    print(tabla)  # Imprime la tabla con los tiempos de ejecución para cada cantidad de procesos


Resultados para sumar los primeros 100000000 enteros con diferentes números de procesos:

   Número de procesos  Tiempo secuencial (s)  Tiempo paralelo (s)
0                   1               0.715842             0.808573
1                   2               0.708581             0.370062
2                   3               0.710627             0.259320
3                   4               0.707032             0.212240
4                   5               0.715834             0.172038
5                   6               0.837411             0.252338
6                   7               0.707846             0.178600
7                   8               0.799508             0.200493


#### Tabla comparativa 
![Texto alternativo](https://github.com/JojhanPerezArroyave/TallerRepasoPython/blob/main/Captura%20de%20pantalla%202024-05-14%20185218.png?raw=true)

- De la imagen podemos concluir que las bondades de la programación paralela son mayormente vistas con grandes volumenes de información, debido a que como podemos ver en la tabla con un n de 10.000 y 1.000.000 hacerlo secuencialmente puede ser más eficiente, pero si pasamos a 10.000.000 y 100.000.000 ya el hacer la operación en paralelo disminuye los tiempos a incluso menos de la mitad del tiempo secuencial, a excepción del momento de hacerlo con un unico proceso.

#### 5. Realizar lo mismo que en (4) para OpenMP.

In [157]:
%%writefile ./multiprocessing_openMp.c

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int main(int argc, char **argv) {
    int N = 100000000; // Número de enteros a sumar
    int max_procesos = 8; // Número máximo de procesos

    // Tabla para almacenar los tiempos de ejecución secuenciales y paralelos
    printf("Resultados para sumar los primeros %d enteros:\n", N);
    printf("%-20s%-20s%-20s\n", "Número de procesos", "Tiempo secuencial", "Tiempo paralelo");

    // Bucle para iterar sobre diferentes números de procesos
    for (int num_procesos = 1; num_procesos <= max_procesos; num_procesos++) {
        // Suma secuencial
        double inicio_secuencial = omp_get_wtime(); // Tiempo de inicio secuencial
        int suma_secuencial = 0;
        // Bucle para realizar la suma secuencial
        for (int i = 1; i <= N; i++) {
            suma_secuencial += i;
        }
        double fin_secuencial = omp_get_wtime(); // Tiempo de finalización secuencial
        double tiempo_secuencial = fin_secuencial - inicio_secuencial;

        // Suma paralela
        double inicio_paralelo = omp_get_wtime(); // Tiempo de inicio paralelo
        int suma_paralela = 0;
        // Bucle paralelo para realizar la suma paralela
        #pragma omp parallel for num_threads(num_procesos) reduction(+:suma_paralela)
        for (int i = 1; i <= N; i++) {
            suma_paralela += i;
        }
        double fin_paralelo = omp_get_wtime(); // Tiempo de finalización paralelo
        double tiempo_paralelo = fin_paralelo - inicio_paralelo;

        // Imprimir resultados de tiempo para la configuración actual
        printf("%-20d%-20f%-20f\n", num_procesos, tiempo_secuencial, tiempo_paralelo);
    }

    return 0;
}



Overwriting ./multiprocessing_openMp.c


In [158]:
!gcc multiprocessing_openMp.c -o multiprocessing_openMp -fopenmp
!./multiprocessing_openMp

Resultados para sumar los primeros 100000000 enteros:
Número de procesos Tiempo secuencial   Tiempo paralelo     
1                   0.075081            0.069587            
2                   0.075466            0.035624            
3                   0.073902            0.024846            
4                   0.075731            0.020150            
5                   0.075147            0.017787            
6                   0.074741            0.016726            
7                   0.074488            0.017105            
8                   0.075950            0.015200            


#### Tabla comparativa 
![Texto alternativo](https://github.com/JojhanPerezArroyave/TallerRepasoPython/blob/main/Captura%20de%20pantalla%202024-05-14%20202842.png?raw=true)

- Podemos ver que sucedio algo muy similar a lo que pasó con multiprocessing pero esta vez los cambios de hacerlo en paralelo se empezarón a ver desde n igual a 1000000 y si hacemos una especie de comparativa esta paralelización automatica de openMP reultó más efectiva.